# CW2.3:  Compiler Back End for FUNC

This week, you will extend the code generator with function calls.

Again, at the end of this file you'll find example program you can test your programs with. 
**You will want to write additional tests for intermediate steps.**

## The Target Language 

We will again use the ``FUNC`` language from CW 2.1 and CW 2.2, see there for a description. 

In this part, you will extend the basic solution with function calls.
You will want to copy over your solution from CW 2.2.

Here is additional information on the requirements for function calls:

- You can use ``$t8`` and ``$t9`` for intermediate computations as we have done for SIMP.
- The code in the ``main()`` function should be executed (using the ``start_:`` label in MIPS). You have to put ``.global _start`` at the beginning of the assembly code to ensure that the assembler finds this label.
    - All other functions can only be reached by calling them from ``main`` (possibly indirectly via other functions).
    - ``main`` cannot be called by any functions (including ``main``).
- You can assume that there are sufficient registers to hold all variables in a function.
    - Registers should be allocated in the order variables are introduced, starting with ``$s0/$t0``
    - Use the ``$s0-$s7`` registers to store variables of the ``main()`` function. 
    - Use the ``$t0-$t7`` registers for the variables of other functions.
    - There are no global variables.
- Use ``$a0-$a3`` to store function arguments. You may assume that there are not more than 4 arguments.
- Use ``$v0`` to store the return value. The variable declared in the optional return statement at the end of each method should be returned in ``$v0``.
- You are allowed to define an arbitrary number of additional functions - but please use the high-level functions for compilation.

The appendix (bottom of this document) contains an example of generated code which you will want to ensure that you understand before you start coding.

**Marking Scheme**. 
You will get 6/10 points for developing a solution which supports basic function calls (but no nested/recursive calls). 
This means you need to implement a suitable register-based protocol where ``$a0-$a3`` is used for arguments, and ``$v0`` for the result. You can assume a maximum of 4 arguments. Remember, you should use the ``$t0-$t7`` registers for local variables (except for main() where ``$s0-$s7`` are used).

You will get the remaining 4/10 points if your solution also supports recursive and nested calls. To achieve this, you need to develop a stack frame. As all variables are in registers, (when required) you will need to push a stack frame with all caller-save registers before the call is made and pop them back to the correct registers when control is returned to the caller


### Additional Hints 

*The assignment is doable without the following hints; but if you need an additional starting point, the following contains a more detailed description on what to do.*

You will want to start to ensure that you 
- understand the details on the slides about function calls. 
- understand the example generated code in the appendix with annotations and understand each of the instructions.

Compared to the basic solution, you will have then to implement the following features: 
- When compiling expressions, you have to add the general case of ``App (s, es)``. In this case, you have to: 
    - Push the stack frame with all caller-save registers. You might want to use the function ``all_registers :: int Env.t -> register list`` which returns all registers currently registered in the environment.
    - Ensure that the expressions ``es`` are compiled into the argument registers ``$a0-$a3``. (Recall that you can assume that there are never more than 4 arguments.) 
        - Hint: You might want to use an additional function ``compile_exps env (rs : register list) (es : exp list) : code`` for doing so, mutually recursive with ``compile_exp`` (``and compile_exps env (rs : register list) (es : exp list) : code = match rs, es with ... ``).
        
    - Register the return address and jump to the label of the subroutine (which is responsible to bring you back to the next instruction).
    - Pop the stack frame.
    - Move the value at ``$v0`` into the goal register.
- When using the argument registers for ``syscall``, you have to push/pop ``$a0`` to ensure you are not overwriting the arguments.
- ``compile_method`` has now also to support arbitrary methods. In particular, this means that 
    - You have to add a label for the subroutine at the beginning of the code of your method.
    - ``compile_dcls`` has to distinguish whether variables are declared in the ``main()`` function or from another function. The pre-defined function ``declare_local_var: int Env.t -> string -> int Env.t`` behaves similar to ``declare_var`` but adds a variable to ``$t0-$t7`` instead of ``$s0-$s7``.
    - You require to declare the argument registers at the beginning of a method; you might want to write a new function for this.
    - You need to jump back to the return address ``$ra`` at the end of a not-main method. 

Below is the abstract grammar for the language you've seen before.

In [ ]:
type exp = Numb of int | Id of string | App of string * exp list

type bop = Less | LessEq | Eq | NEq 
type cond = C of bop * exp * exp

type statement =
  Assign of string * exp
| Read of string 
| Write of exp 
| If of cond * statement list
| Ite of cond * statement list * statement list
| While of cond * statement list

type mmethod = M of string (* name of function *)
                * string list (* arguments *)
                * string list (* declarations *) 
                * statement list (* function body *)
                * string option (* possible return value value *)

type program = P of mmethod list

# Definitions

In [ ]:
(* We represent registers as numbers. Registers are represented by 0 to 31. *)
type register = int

(* Value returned by a subroutine *)
let v0 : register = 2 
let v1 : register = 3 

(* Arguments to subroutine *)
let a0 : register = 4 
let a1 : register = 5
let a2 : register = 6
let a3 : register = 7

(* Temporary registers *)
let t0 : register = 8
let t1 : register = 9
let t2 : register = 10
let t3 : register = 11
let t4 : register = 12
let t5 : register = 13
let t6 : register = 14
let t7 : register = 15

(* Saved registers *)
let s0 : register = 16
let s1 : register = 17 
let s2 : register = 18 
let s3 : register = 19 
let s4 : register = 20 
let s5 : register = 21
let s6 : register = 22 
let s7 : register = 23 

(* Temporary registers $t8 and $t9 will be used for intermediate results. *)
let t8 : register = 24 (* $t8 *)
let t9 : register = 25 (* $t9 *)

let (sp : register) = 29 (* stack pointer *)
let (fp : register) = 30 (* frame pointer *)
let (ra : register) = 31 (* return address *)

(* We represent instructions as an abstract data type. *)

type label = string

type instruction =  Add of register * register * register (* add $1, $2, $3 - $1 = $2 + $3 *)
                   | Sub of register * register * register (* sub $1, $2, $3; $1 = $2 - $3 *)
                   | Addi of register * register * int (* addi $1, $2, 100 - $1 = $2 + 100, immediate means a constant number  *)
                   | Addiu of register * register * int (* addi $1, $2, 100 - $1 = $2 + 100, values treated as unsigned, immediate means a constant number  *)
                   | Mul of register * register * register (* mul $1, $2, $3 - $1 = $2 * $3, without overflow, result is only 32 bits *)
                   | Div of register * register (* div $2, $3 - $hi,$low=$2/$3, Remainder stored in special register hi, Quotient stored in special register lo   *)
                   | And of register * register * register (* and $1, $2, $3 - $1 = $2 & $3, bitwise AND *)
                   | Or of register * register * register (* or $1, $2, $3 - $1 = $2 | 100, bitwise OR *)
                   | Andi of register * register * int (* andi $1, $2, 100 - $1 = $2 & 100, bitwise AND with immediate value  *)
                   | Ori of register * register * int (* ori $1, $2, 100 - $1 = $2 | 100, bitwise OR with immediate value  *)
                   | Lw of register * int * register (* lw $1, 100 ($2) - load word, $1 = Memory[$2 + 100], copy from memory to register *)
                   | Sw of register * int * register (* sw $1, 100 ($2) - store word, Memory[$2 + 100] = $1, copy from register to memory *)
                   | La of register * label (* $1 = Address of label *) 
                   | Li of register * int (* li $1, 100 - Loads immediate value into register *)
                   | Move of register * register (* move $1,$2 - $1 = $2, Copy from register to register *)
                   | Mfhi of register (* mfhi $2, $2 = hi, copy from special register hi to general register *)
                   | Mflo of register (* mflo $2, $2 = lo, copy from special register lo to general register *)
                   | Label of label 
                   | Beq of register * register * string (* beq $1, $2, l - if ($1 == $2) go to label l *)
                   | Bne of register * register * string (* bne $1, $2, l - if ($1 != $2) go to label l *)
                   | Bgt of register * register * string (* bgt $1, $2, l - if ($1 > $2) go to label l *)
                   | Blt of register * register * string (* blt $1, $2, l - if ($1 < $2) go to label l *)
                   | Bge of register * register * string (* bge $1, $2, l - if ($1 >= $2) go to label l *)
                   | Ble of register * register * string (* ble $1, $2, l - if ($1 <= $2) go to label l *)                  
                   | J of label (* j l, go to label l, jumps to target address *)
                   | Jr of register (* jump register, jr $1, go to address stored in $1 *)
                   | Jal of label (* jump and link, e.g. jal l - $ra=PC+4; go to label l - used when making procedure call. This saves the return address in $ra.  *)
                   | SysCall 
                   | Verbatim of string (* Produce the given string verbatim in the assembly output *)
                   
type code = instruction list

let print_register (r : register) = 
    match r with 
    | 2 | 3 -> "$v" ^ (string_of_int (r - v0)) 
    | 4 |5 |6 | 7 -> "$a" ^ string_of_int (r - a0)
    | 8|9|10|11|12|13|14|15 -> "$t" ^ string_of_int (r - t0) 
    | 16|17|18|19|20|21|22|23 -> "$s" ^ string_of_int (r - s0)
    | 24 -> "$t8"
    | 25 -> "$t9"
    | 29 -> "$sp"
    | 30 -> "$fp"
    | 31 -> "$ra"
    | _ -> "$" ^ string_of_int r

let print_instruction (i : instruction) = match i with 
    | Add (r1, r2, r3) -> "add " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Sub (r1, r2, r3) -> "sub " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Addi (r1, r2, i) -> "addi " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ string_of_int i
    | Addiu (r1, r2, i) -> "addiu " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ string_of_int i
    | Mul (r1, r2, r3) -> "mul " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Div (r1, r2) -> "div " ^ print_register r1 ^ ", " ^ print_register r2
    | Beq (r1, r2, l) ->  "beq " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Bne (r1, r2, l) ->  "bne " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Bgt (r1, r2, l) ->  "bgt " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Blt (r1, r2, l) ->  "blt " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Bge (r1, r2, l) ->  "bge " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Ble (r1, r2, l) ->  "ble " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Li (r, i) -> "li " ^ print_register r ^ ", " ^ string_of_int i
    | Lw (r1, o, r2) -> "lw " ^ print_register r1 ^ ", " ^ string_of_int o ^ "(" ^ print_register r2 ^ ")" 
    | La (r, l) -> "la " ^ print_register r ^ ", " ^  l
    | Sw (r1, o, r2) -> "sw " ^ print_register r1 ^ ", " ^ string_of_int o ^ "(" ^ print_register r2 ^ ")" 
    | Move (r1, r2) -> "move " ^ print_register r1 ^ ", " ^ print_register r2
    | Mfhi r -> "mfhi "^ print_register r
    | Mflo r -> "mflo "^ print_register r
    | And (r1, r2, r3) -> "and " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Andi (r1, r2, r3) -> "andi " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Or (r1, r2, r3) -> "or " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Ori (r1, r2, r3) -> "ori " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | SysCall -> "syscall"
    | Label l -> l ^ ":"
    | J label -> "j " ^ label 
    | Jr r -> "jr " ^ print_register r
    | Jal label -> "jal " ^ label
    | Verbatim s -> s
   
let rec print_code (c : code) : unit = match c with 
    | [] -> ()
    | c :: cs -> (print_endline (print_instruction c); print_code cs)
    
exception EEnv of string

let maxreg = 23

module Env = Map.Make (String)

(*  Function that finds the largest register in the environment. *)
let find_max_register (env : int Env.t) = 
    Env.fold (fun _ a b -> max a b) env (s0 - 1) 


(* Declaring a variable: 
  - When trying to declare a variable, and there are too many registers already 
    reserved, throw an exception. 
  - Else: Assign to variable x the largest register number + 1.
*)
let declare_var (env: int Env.t) (x:string) : int Env.t = 
    if (find_max_register env) >= maxreg
     then raise (EEnv "Too many variables")
     else Env.add x (1 + find_max_register env) env
     
exception E of string

(* Pushing the content of register r to the stack *)
let push (r : register) : code = [Addiu (sp, sp, -4);
                                  Sw (r, 0, sp)]

(* Popping the stack into register r *)
let pop (r : register) : code = [Lw (r, 0, sp);
                                 Addiu (sp, sp, 4)]
                                 
                                 
let counter : int ref = {contents = 0}

let next_val = 
    fun () ->
      counter := (!counter) + 1;
      !counter;;                                 

Here are some additional helper functions which might be useful:
- ``all_registers`` yields a list of all registers used by an environment. 
- ``declare_local_var`` behaves similar to ``declare_var``, but assigns the smallest unused **local register** (that is, ``$t0`` to ``$t7``) to a variable ``x``. 

In [ ]:
(* The following function yields a list of all registers used by an environment env *)
let all_registers env = Env.fold (fun k x xs -> x :: xs) env []

(* Variants of declare_var for local variables. *)
let maxlocalreg = 15

(*  Function that finds the largest local register in the environment. *)
let find_max_local_register (env : int Env.t) = 
    Env.fold (fun _ a b -> if a > maxlocalreg then b else max a b) env (t0 - 1) 

(* Declaring a local variable: 
  - When trying to declare a local variable, and there are too many registers already 
    reserved, throw an exception. 
  - Else: Assign to variable x the largest local register number + 1.
*)
let declare_local_var (env: int Env.t) (x:string) : int Env.t = 
    if (find_max_local_register env) >= maxlocalreg
     then raise (EEnv "Too many variables")
     else Env.add x (1 + find_max_local_register env) env

## Your Code

Fill out the gaps below.

In [ ]:
let argument_registers = [a0; a1; a2; a3]

(* TODO: Code for pushing/popping the stack frame. 
You might want to use the function all_registers, where all_registers env yields all registers 
at the momement registers in the environment.
*)
let push_stack_frame  env : code =
    [Verbatim "# Start pushing stack frame"] 
    @ (* TODO : Fill out *) []
    @ [Verbatim "# End pushing stack frame"]

let pop_stack_frame env: code = 
    [Verbatim "# Start popping stack frame"] 
    @ (* TODO: Fill out *) []
    @ [Verbatim "# End popping stack frame"]


(* Compiling code that saves the value of an expression e into register r. 

    CW 2.3 requires you only to change the last case.
   The basic function calls solution can ignore nested and recursive calls and hence the stack frame.
*)
let rec compile_exp env (r : register) (e : exp)  : code = match e with 
    | Id s -> (* TODO: Fill out. *)
    | Numb n -> (* TODO: Fill out. *)
    | App ("plus", e1 :: e2 :: []) -> (* TODO: Fill out. *)
    | App ("minus", e1 :: e2 :: []) -> (* TODO: Fill out. *)
    | App ("times", e1 :: e2 :: []) -> (* TODO: Fill out. *)
    | App ("divide", e1 :: e2 :: []) -> (* TODO: Fill out. *)
    | App (s, es) -> raise (E "CW 2.3: Implement general function calls. ")

(* Generate code that saves the values of a list of expressions into a list of registers rs, typically 
the argument registers. *)
and compile_exps env (rs : register list) (es : exp list) : code = match rs, es with 
    (* TODO: Implement for CW 2.3 *)

(* The following code mainly stays the same - but for the cases in which $a0 is used for syscalls. *)

(* Generate code for a single command. *)
let rec compile_cmd env (c : statement) : code = match c with 
    (* TODO: Fill out. *)

(* Generate code for a list of commands. *)
and compile_cmds env (cs : statement list) : code = match cs with 
    (* TODO: Fill out. *)

In [ ]:
(* The f is the function name for which declarations are called 
- this could be either "main" or any other function. 
Add case where f is not "main" using the function declare_local_var. *)
let rec compile_dcls env (f : string) dcls = match dcls with 
    | [] -> env 
    | dcl :: dcls' -> let env' = if f = "main" then declare_var env dcl else declare_local_var env dcl
                      in compile_dcls env' f dcls'

(* Generate code for a method - Add second case for arbitrary functions compared to CW 2.2 *)
let rec compile_method (m : mmethod) : code = match m with 
 | M ("main", [], dcls, stms, None) -> (* IDENTICAL TO CW 2.2 *)
 | M (name, args, dcls, stms, ret) -> raise (E "General methods to be implemented.")


(* Generate code for a list of methods - Should be identical to CW 2.2 *)
let rec compile_methods (ms : mmethod list) : code = match ms with 
    (* TODO: Fill out. *)

(* Generate code for a whole program - Should be identical to CW 2.2 *)
let compile_program (p : program) = match p with 
    | P ms ->      
        (* TODO: Fill out. *)

## Example Programs 

Below are the three example programs from the appendix as an AST. 
You can see example MIPS output for the full solution in the appendix (code for ``parsed_basic`` is in CW 2.2).

In [ ]:
let parsed_basic = 
  P
    [M ("main", [], ["inp"; "res"],
      [Read "inp"; Assign ("res", Numb 0);
       While (C (Less, Numb 0, Id "inp"),
        [Assign ("res", App ("plus", [Id "res"; Id "inp"]));
         Assign ("inp", App ("minus", [Id "inp"; Numb 1]))]);
       Write (Id "res")],
      None)];;

print_code (compile_program parsed_basic)

In [ ]:
let parsed_basic_function = 
P
    [M ("sum", ["inp"], ["res"],
      [Assign ("res", Numb 0);
       While (C (Less, Numb 0, Id "inp"),
        [Assign ("res", App ("plus", [Id "res"; Id "inp"]));
         Assign ("inp", App ("minus", [Id "inp"; Numb 1]))])],
      Some "res");
     M ("main", [], ["inp"; "res"],
      [Read "inp"; Assign ("res", App ("sum", [Id "inp"])); Write (Id "res")],
      None)];;

print_code (compile_program parsed_basic_function)

In [ ]:
let parsed_nested = 
P
    [M ("sum", ["inp"], ["tmp"; "res"],
      [Ite (C (Eq, Id "inp", Numb 0), [Assign ("res", Id "inp")],
        [Assign ("tmp", App ("sum", [App ("minus", [Id "inp"; Numb 1])]));
         Assign ("res", App ("plus", [Id "tmp"; Id "inp"]))])],
      Some "res");
     M ("main", [], ["inp"; "res"],
      [Read "inp"; Assign ("res", App ("sum", [Id "inp"])); Write (Id "res")],
      None)];;

print_code (compile_program parsed_nested)

## Appendix: Examples of FUNC code & generated MIPS code

### Basic Solution 

The following code should be supported in the basic solution:
```
method main() vars inp, res
begin
    read inp;
    res:=0;
    while less(0,inp)
    begin
        res := plus(res,inp);
        inp := minus(inp,1); 
    endwhile;
    write res;
endmethod;
```

The following MIPS code is a possible solution: 

```
.global _start
.set noreorder
.data
sinp:
.asciiz "INPUT>"  
.text
_start:
li $v0, 4
addiu $sp, $sp, -4
sw $a0, 0($sp)
la $a0, sinp
syscall
li $v0, 5
syscall
move $s0, $v0
lw $a0, 0($sp)
addiu $sp, $sp, 4
li $s1, 0
WLOOP4:
li $t8, 0
addiu $sp, $sp, -4
sw $t8, 0($sp)
move $t9, $s0
lw $t8, 0($sp)
addiu $sp, $sp, 4
bge $t8, $t9, WEND4
move $t8, $s1
addiu $sp, $sp, -4
sw $t8, 0($sp)
move $t9, $s0
lw $t8, 0($sp)
addiu $sp, $sp, 4
add $s1, $t8, $t9
move $t8, $s0
addiu $sp, $sp, -4
sw $t8, 0($sp)
li $t9, 1
lw $t8, 0($sp)
addiu $sp, $sp, 4
sub $s0, $t8, $t9
j WLOOP4
WEND4:
li $v0, 1
addiu $sp, $sp, -4
sw $a0, 0($sp)
move $a0, $s1
syscall
lw $a0, 0($sp)
addiu $sp, $sp, 4
li $v0, 10
syscall
```

It should sum up all numbers from 0 to the given number. For example, if it reads ``3`` then it should print ``6`` (computing ``3+2+1``). Note that your solution does not have to be identical as long as the program produces the same answer and satisfies the specification in this document. 

## Basic Function Calls

The following program illustrates an example that should work with basic function calls. It uses a computation that sums up the result in a separate function ``sum``: 

```
method sum(inp) vars res
    begin
    res:=0;
    while less(0,inp)
    begin
        res := plus(res,inp);
        inp := minus(inp,1);
    endwhile;
    return res;
endmethod;

method main() vars inp,res
    begin
    read inp;
    res := sum(inp);
    write res;
endmethod;
```

The following MIPS code is a correct generation: 

```
.global _start
.set noreorder
.data
sinp:
.asciiz "INPUT>"  
.text
# Subroutine for sum. The environment has to know that inp is $a0.
sum:

# res := 0
li $t0, 0

# start of while condition
WLOOP5:
li $t8, 0
addiu $sp, $sp, -4
sw $t8, 0($sp)
# $a0 corresponds to inp.
move $t9, $a0
lw $t8, 0($sp)
addiu $sp, $sp, 4
bge $t8, $t9, WEND5

# Start of while body.
move $t8, $t0
addiu $sp, $sp, -4
sw $t8, 0($sp)
move $t9, $a0
lw $t8, 0($sp)
addiu $sp, $sp, 4
add $t0, $t8, $t9
move $t8, $a0
addiu $sp, $sp, -4
sw $t8, 0($sp)
li $t9, 1
lw $t8, 0($sp)
addiu $sp, $sp, 4
sub $a0, $t8, $t9
j WLOOP5
WEND5:

# After while body: Move result in $v0 and jump to the return address.
move $v0, $t0
jr $ra

# Code for main function, jumping here because of .global _start.
_start:

# read inp, where inp is in $s0 as it's the first variable in the main function.
li $v0, 4
la $a0, sinp
syscall
li $v0, 5
syscall
move $s0, $v0

# res := sum(inp) - loading argument into $a0, and then jump to subroutine sum. At the end, move the result into the register for res, i.e. $s1.
move $a0, $s0
jal sum
move $s1, $v0

# write res;
li $v0, 1
move $a0, $s1
syscall

# end MIPS program
li $v0, 10
syscall
```

Your solution does not have to be identical as long as the program produces the same answer and satisfies the specification in this document. 


## Nested/Recursive Calls

The following program illustrates support for nested and function calls. It does the same as the previous one, but computes the result using recursion:

```
method sum(inp) vars tmp, res
begin
    if eq(inp,0) then
        res := inp;
    else
        tmp := sum(minus(inp,1));
        res := plus(tmp,inp);
    endif;
    return res;
endmethod;

method main() vars inp,res
begin
    read inp;
    res := sum(inp);
    write res;
endmethod;
```

The following slightly annotated MIPS code is a possible solution: 
```
.global _start
.set noreorder
.data
sinp:
.asciiz "INPUT>"  
.text

# Subroutine for sum; the environment has to know that inp is in register $a0.
sum:

# eq (inp, 0)
move $t8, $a0
addiu $sp, $sp, -4
sw $t8, 0($sp)
li $t9, 0
lw $t8, 0($sp)
addiu $sp, $sp, 4
bne $t8, $t9, IFFALSE3

# res := inp, where $res is in $t1 (second variable of non-main function) and $inp is in $a0 (first argument)
move $t1, $a0
j IFEND3

# Else case.
IFFALSE3:
# Start pushing stack frame as we have to call sum (minus (inp, 1)); pushing the return address, all caller-save registers, all arguments
addiu $sp, $sp, -4
sw $ra, 0($sp)
addiu $sp, $sp, -4
sw $t0, 0($sp)
addiu $sp, $sp, -4
sw $t1, 0($sp)
addiu $sp, $sp, -4
sw $a0, 0($sp)
# End pushing stack frame

# moving min (inp, 1) into $a0
move $t8, $a0
addiu $sp, $sp, -4
sw $t8, 0($sp)
li $t9, 1
lw $t8, 0($sp)
addiu $sp, $sp, 4
sub $a0, $t8, $t9

# jumping to subroutine
jal sum

# Start popping stack frame
lw $a0, 0($sp)
addiu $sp, $sp, 4
lw $t1, 0($sp)
addiu $sp, $sp, 4
lw $t0, 0($sp)
addiu $sp, $sp, 4
lw $ra, 0($sp)
addiu $sp, $sp, 4
# End popping stack frame

# Moving result into $tmp, i.e. $t0
move $t0, $v0

# res := plus(tmp,inp)
move $t8, $t0
addiu $sp, $sp, -4
sw $t8, 0($sp)
move $t9, $a0
lw $t8, 0($sp)
addiu $sp, $sp, 4
add $t1, $t8, $t9

# After if part
IFEND3:

# return res; - so moving result ($t1) into result register $v0.
move $v0, $t1
# Jump back to return address.
jr $ra


# Code for main function, jumping here because of .global _start.
_start:

# read inp;
li $v0, 4
# pushing $a0 as it might be an argument
addiu $sp, $sp, -4
sw $a0, 0($sp)
la $a0, sinp
syscall
li $v0, 5
syscall
move $s0, $v0
# popping $a0
lw $a0, 0($sp)
addiu $sp, $sp, 4

# res := sum(inp);

# Start pushing stack frame
addiu $sp, $sp, -4
sw $ra, 0($sp)
# End pushing stack frame

# move inp into argument slot.
move $a0, $s0

jal sum

# Start popping stack frame
lw $ra, 0($sp)
addiu $sp, $sp, 4
# End popping stack frame

# Moving result into res.
move $s1, $v0

# write res;
li $v0, 1
# pushing $a0 as it might be an argument
addiu $sp, $sp, -4
sw $a0, 0($sp)
move $a0, $s1
syscall
# popping $a0
lw $a0, 0($sp)
addiu $sp, $sp, 4

# End MIPS program
li $v0, 10
syscall
```

Again, your solution does not have to be identical as long as the program produces the same answer and satisfies the specification in this document. 